In [1]:
import numpy as np
import matplotlib.pyplot as plt
from TLE_ import*
from TLE_file import*

In [2]:
TLE_objs = TLE_obj_from_file("TLE.txt")
print(TLE_objs[0])


TLE data for ISS (ZARYA)
 
NORAD ID: 25544U
Epoch: 24136.79884226
Inclination: 51.6378
Right ascension of the ascending node: 115.9329
Eccentricity: 0003286
Argument of perigee: 171.4661
Mean anomaly: 259.6570
Mean motion: 15.51363823453527

